In [291]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MultiLabelBinarizer

In [292]:
rg_citations = pd.read_csv('../ResearchGate/research_gate_citations.csv', sep='|')
rg_profiles = pd.read_csv('../ResearchGate/research_gate_profiles.csv', sep='|')

In [233]:
rg_citations

,faculty,citation,title,type,abstract,book,journal,date,collabs,publisher,...,series,volume,issue,chapter,pages,doi,issn,isbn,handle,url
0,Yue (Jeff) Zhang,"Yang, Jing & Zhang, Yue Jeff & Lanting, Cees. ...",Exploring the Impact of QR Codes in Authentica...,Article,The fast growing usage of mobile devices to ac...,NaN,NaN,Wireless Personal Communications,Jing YangYue Jeff ZhangCees J. M. Lanting,NaN,...,NaN,NaN,NaN,NaN,NaN,10.1007/s11277-016-3743-5,NaN,NaN,NaN,https://www.researchgate.net/publication/30890...
1,Yue (Jeff) Zhang,"Yang, Yanni & Zhang, Yue Jeff & Yu Jiang, Yan ...",Leveraging new media applications in response ...,Article,The age of new media brings new challenges and...,NaN,NaN,International Journal of Information Systems a...,Yanni YangYue Jeff ZhangYan Yu JiangYang Shen,NaN,...,NaN,NaN,NaN,NaN,NaN,10.1504/IJISCM.2014.069394,NaN,NaN,NaN,https://www.researchgate.net/publication/27791...
2,Yue (Jeff) Zhang,"Zhan, Hong & Tang, Tian & Zhang, Yue Jeff. (20...",The Reaserch on Characteristics of Knowledge W...,Article,NaN,NaN,NaN,American Journal of Industrial and Business Ma...,Hong ZhanTian TangYue Jeff Zhang,NaN,...,NaN,NaN,NaN,NaN,NaN,10.4236/ajibm.2013.36064,NaN,NaN,NaN,https://www.researchgate.net/publication/27649...
3,Yue (Jeff) Zhang,"Zhang, Yue Jeff & Hsieh, Chang-tseh. (2010). C...",Chinese citizens? Opinions on e-government ben...,Article,E-government is important for China's economic...,NaN,NaN,Electronic Government an International Journal...,Yue Jeff ZhangChang-tseh Hsieh,NaN,...,NaN,NaN,NaN,NaN,NaN,10.1504/EG.2010.030924,NaN,NaN,NaN,https://www.researchgate.net/publication/22008...
4,Yue (Jeff) Zhang,"Zhang, Yue Jeff & Richard Ye, Li & Lin, Zhiyan...",Electronic Government in China,Article,This chapter focuses on China’s e-government p...,NaN,NaN,· August 2009,Yue Jeff ZhangLi Richard YeZhiyang LinQuan Lin,NaN,...,NaN,NaN,NaN,NaN,NaN,10.1007/978-0-387-77743-6_20,NaN,NaN,NaN,https://www.researchgate.net/publication/25126...
5,Yue (Jeff) Zhang,"Zhang, Yue Jeff & A. Driscoll, Donna & J. Laza...",What affects travellers' preference of e-trave...,Article,The world wide web is re-shaping the 'maps' of...,NaN,NaN,International Journal of Services and Standard...,Yue Jeff ZhangDonna A. DriscollPaul J. Lazaron...,NaN,...,NaN,NaN,NaN,NaN,NaN,10.1504/IJSS.2006.010471,NaN,NaN,NaN,https://www.researchgate.net/publication/22027...
6,Yue (Jeff) Zhang,"Wu, Jiinpo & Chen, Charlie & Zhang, Yue Jeff &...",Virtual Group Learning Effectiveness via eLear...,Conference Paper,Online learning provides the potential for mor...,NaN,NaN,· January 2005,Jiinpo WuCharlie C. ChenYue Jeff ZhangDonald L...,NaN,...,NaN,NaN,NaN,NaN,NaN,DBLP,NaN,NaN,NaN,https://www.researchgate.net/publication/22089...
7,Yue (Jeff) Zhang,"Lu Wang, Cheng & Zhang, Yue Jeff & Richard Ye,...",Subscription to fee-based online services: Wha...,Article,"Increasingly, Web-based content or service sit...",NaN,NaN,· January 2005,Cheng Lu WangYue Jeff ZhangLi Richard YeDat Ng...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.researchgate.net/publication/24240...
8,Yue (Jeff) Zhang,"Chen, Jim & Zhang, Yue Jeff & Pearson, J & Che...",An Exploratory Study of E-MBA Students' Percep...,Article,China's acceptance into the World Trade Organi...,NaN,NaN,Journal of Internet Commerce 3(4):1-18,Jim Q ChenYue Jeff ZhangJ. Michael PearsonJin ...,NaN,...,NaN,NaN,NaN,NaN,NaN,10.1300/J179v03n04_01,NaN,NaN,NaN,https://www.researchgate.net/publication/25436...
9,Yue (Jeff) Zhang,"Chen, Jim & E. Lee, Ted & Zhang, Ruidong & Zha...",Systems Requirements for Organizational Learning,Article,The organizational learning support systems (O...,NaN,NaN,Communications of the ACM 46(12):73-78,Jim Q ChenTed E. LeeRuidong ZhangYue Jeff Zhang,NaN,...,NaN,NaN,NaN,NaN,NaN,10.1145/953460.953462,NaN,NaN,NaN,https://www.researchgate.net/profile/Yue_Jeff_...


In [234]:
rg_profiles

,faculty,college,department,bio,personal,academic,research,orcid,scopus
0,Yue (Jeff) Zhang,College of Business and Economics,Department of Accounting,NaN,NaN,NaN,E-Government,NaN,NaN
1,Diane Lewis-Goldstein,Health & Human Development,Department of Family and Consumer Studies,NaN,NaN,NaN,Counseling Self-efficacy,NaN,NaN
2,Tracie Tung,Health & Human Development,Department of Family and Consumer Studies,NaN,NaN,NaN,Academic Writing Qualitative Analysis Research...,NaN,NaN
3,Yoko Mimura,Health & Human Development,Department of Family and Consumer Studies,NaN,NaN,NaN,Statistical Analysis Logistic Regression SAS S...,NaN,NaN
4,Elizabeth Sussman,Health & Human Development,Department of Family and Consumer Studies,NaN,NaN,NaN,Nutrition Clinical Nutrition Nephrology,NaN,NaN
5,Setareh Torabian-Riasati,Health & Human Development,Department of Family and Consumer Studies,NaN,NaN,NaN,Clinical Nutrition Registered Dietitian,NaN,NaN
6,Rosalia Garcia-Torres,Health & Human Development,Department of Family and Consumer Studies,NaN,NaN,NaN,Food Science and Technology Food Chemistry Foo...,NaN,NaN
7,Rebekah Child,Health & Human Development,Department of Nursing,NaN,NaN,NaN,Advertising Emergency Nursing Social Media Soc...,NaN,NaN
8,Laurie Gelardi,Health & Human Development,Department of Nursing,NaN,NaN,NaN,NaN,NaN,NaN
9,Marianne Hattar-Pollara,Health & Human Development,Department of Nursing,NaN,NaN,NaN,Cultural Studies Qualitative Analysis Culture ...,NaN,NaN


In [235]:
shapiro_citations = rg_citations[rg_citations['faculty'] == 'Bruce Shapiro']

In [236]:
citations_with_abstracts = shapiro_citations[shapiro_citations['abstract'].notnull()]

In [237]:
citations_with_abstracts

,faculty,citation,title,type,abstract,book,journal,date,collabs,publisher,...,series,volume,issue,chapter,pages,doi,issn,isbn,handle,url
650,Bruce Shapiro,"Sajjadi, Seyed & Shapiro, Bruce & McKinlay, Ch...",Finding Bottlenecks: Predicting Student Attrit...,Article,With pressure to increase graduation rates and...,NaN,NaN,· May 2017,Seyed SajjadiBruce ShapiroChristopher McKinlay...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.researchgate.net/publication/31678...
651,Bruce Shapiro,"Shapiro, Bruce & Mjolsness, Eric. (2015). Pyce...",Pycellerator: An arrow-based reaction-like mod...,Article,"Motivation: We introduce Pycellerator, a Pytho...",NaN,NaN,Bioinformatics 32(4),Bruce ShapiroEric Mjolsness,NaN,...,NaN,NaN,NaN,NaN,NaN,10.1093/bioinformatics/btv596,NaN,NaN,NaN,https://www.researchgate.net/publication/28329...
652,Bruce Shapiro,"Shapiro, Bruce & Tobin, Cory & Mjolsness, Eric...",Analysis of cell division patterns in the Arab...,Article,The stereotypic pattern of cell shapes in the ...,NaN,NaN,Proceedings of the National Academy of Science...,Bruce ShapiroCory TobinEric MjolsnessElliot M....,NaN,...,NaN,NaN,NaN,NaN,NaN,10.1073/pnas.1502588112,NaN,NaN,NaN,https://www.researchgate.net/publication/27431...
653,Bruce Shapiro,"Shapiro, Bruce & Meyerowitz, Elliot & Mjolsnes...",Using Cellzilla for Plant Growth Simulations a...,Article,Cellzilla is a two-dimensional tissue simulati...,NaN,NaN,Frontiers in Plant Science 4:408,Bruce ShapiroElliot M. MeyerowitzEric Mjolsness,NaN,...,NaN,NaN,NaN,NaN,NaN,10.3389/fpls.2013.00408,NaN,NaN,NaN,https://www.researchgate.net/publication/25805...
655,Bruce Shapiro,"Nikolaev, Sergey & Zubairova, Ulyana & Penenko...",Doklady Biological Sciences,Article,(translation of original Russian publication.),NaN,NaN,Doklady Biological Sciences 452:316-319,Sergey NikolaevUlyana ZubairovaAlexey PenenkoE...,NaN,...,NaN,NaN,NaN,NaN,NaN,10.7868/S0869565213280268,NaN,NaN,NaN,https://www.researchgate.net/publication/26613...
656,Bruce Shapiro,"Shapiro, Bruce & Jönsson, Henrik & Sahlin, Pat...",Tessellations and Pattern Formation in Plant G...,Article,The shoot apical meristem (SAM) is a dome-shap...,NaN,NaN,· September 2012,Bruce ShapiroHenrik JönssonPatrick SahlinMarcu...,NaN,...,NaN,NaN,NaN,NaN,NaN,arXiv,NaN,NaN,NaN,https://www.researchgate.net/publication/23083...
657,Bruce Shapiro,"Shapiro, Bruce & Jönsson, Henrik & Sahlin, Pat...",Tessellations and Pattern Formation in Plant G...,Conference Paper,The shoot apical meristem (SAM) is a dome-shap...,NaN,NaN,· September 2012,Bruce ShapiroHenrik JönssonPatrick SahlinMarcu...,NaN,...,NaN,NaN,NaN,NaN,NaN,The,NaN,NaN,NaN,https://www.researchgate.net/publication/31138...
658,Bruce Shapiro,"Shapiro, Bruce & Finney, Andrew & Hucka, Micha...",SBML models and MathSBML,Chapter,"MathSBML is an open-source, freely downloadabl...",NaN,NaN,· January 2007,Bruce ShapiroAndrew FinneyMichael HuckaBenjami...,NaN,...,NaN,NaN,NaN,NaN,NaN,book,NaN,NaN,NaN,https://www.researchgate.net/publication/22660...
659,Bruce Shapiro,"S Najdi, Tarek & Yang, Chin-Rang & Shapiro, Br...",Application of a Generalized MWC Model for the...,Article,In our effort to elucidate the systems biology...,NaN,NaN,Journal of Bioinformatics and Computational Bi...,Tarek S NajdiChin-Rang YangBruce ShapiroG. Wes...,NaN,...,NaN,NaN,NaN,NaN,NaN,10.1109/CSB.2005.15,NaN,NaN,NaN,https://www.researchgate.net/publication/69647...
660,Bruce Shapiro,"Finney, Andrew & Hucka, Michael & J Bornstein,...",Software Infrastructure for Effective Communic...,Article,"Until recently, the majority of computational ...",NaN,NaN,· March 2006,Andrew FinneyMichael HuckaBenjamin J Bornstein...,NaN,...,NaN,NaN,NaN,NaN,NaN,10.7551/mitpress/9780262195485.003.0017,NaN,NaN,NaN,https://www.researchgate.net/publication/26484...


In [238]:
abstracts = np.array(citations_with_abstracts['abstract'].tolist())

In [239]:
stemmer = SnowballStemmer('english')
words = stopwords.words('english')

In [241]:
citations_with_abstracts['cleaned'] = citations_with_abstracts['abstract'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

/home/frostburn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [242]:
citations_with_abstracts

,faculty,citation,title,type,abstract,book,journal,date,collabs,publisher,...,volume,issue,chapter,pages,doi,issn,isbn,handle,url,cleaned
650,Bruce Shapiro,"Sajjadi, Seyed & Shapiro, Bruce & McKinlay, Ch...",Finding Bottlenecks: Predicting Student Attrit...,Article,With pressure to increase graduation rates and...,NaN,NaN,· May 2017,Seyed SajjadiBruce ShapiroChristopher McKinlay...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.researchgate.net/publication/31678...,with pressur increas graduat rate reduc time d...
651,Bruce Shapiro,"Shapiro, Bruce & Mjolsness, Eric. (2015). Pyce...",Pycellerator: An arrow-based reaction-like mod...,Article,"Motivation: We introduce Pycellerator, a Pytho...",NaN,NaN,Bioinformatics 32(4),Bruce ShapiroEric Mjolsness,NaN,...,NaN,NaN,NaN,NaN,10.1093/bioinformatics/btv596,NaN,NaN,NaN,https://www.researchgate.net/publication/28329...,motiv we introduc pyceller python librari read...
652,Bruce Shapiro,"Shapiro, Bruce & Tobin, Cory & Mjolsness, Eric...",Analysis of cell division patterns in the Arab...,Article,The stereotypic pattern of cell shapes in the ...,NaN,NaN,Proceedings of the National Academy of Science...,Bruce ShapiroCory TobinEric MjolsnessElliot M....,NaN,...,NaN,NaN,NaN,NaN,10.1073/pnas.1502588112,NaN,NaN,NaN,https://www.researchgate.net/publication/27431...,the stereotyp pattern cell shape arabidopsi sh...
653,Bruce Shapiro,"Shapiro, Bruce & Meyerowitz, Elliot & Mjolsnes...",Using Cellzilla for Plant Growth Simulations a...,Article,Cellzilla is a two-dimensional tissue simulati...,NaN,NaN,Frontiers in Plant Science 4:408,Bruce ShapiroElliot M. MeyerowitzEric Mjolsness,NaN,...,NaN,NaN,NaN,NaN,10.3389/fpls.2013.00408,NaN,NaN,NaN,https://www.researchgate.net/publication/25805...,cellzilla two dimension tissu simul platform p...
655,Bruce Shapiro,"Nikolaev, Sergey & Zubairova, Ulyana & Penenko...",Doklady Biological Sciences,Article,(translation of original Russian publication.),NaN,NaN,Doklady Biological Sciences 452:316-319,Sergey NikolaevUlyana ZubairovaAlexey PenenkoE...,NaN,...,NaN,NaN,NaN,NaN,10.7868/S0869565213280268,NaN,NaN,NaN,https://www.researchgate.net/publication/26613...,translat origin russian public
656,Bruce Shapiro,"Shapiro, Bruce & Jönsson, Henrik & Sahlin, Pat...",Tessellations and Pattern Formation in Plant G...,Article,The shoot apical meristem (SAM) is a dome-shap...,NaN,NaN,· September 2012,Bruce ShapiroHenrik JönssonPatrick SahlinMarcu...,NaN,...,NaN,NaN,NaN,NaN,arXiv,NaN,NaN,NaN,https://www.researchgate.net/publication/23083...,the shoot apic meristem sam dome shape collect...
657,Bruce Shapiro,"Shapiro, Bruce & Jönsson, Henrik & Sahlin, Pat...",Tessellations and Pattern Formation in Plant G...,Conference Paper,The shoot apical meristem (SAM) is a dome-shap...,NaN,NaN,· September 2012,Bruce ShapiroHenrik JönssonPatrick SahlinMarcu...,NaN,...,NaN,NaN,NaN,NaN,The,NaN,NaN,NaN,https://www.researchgate.net/publication/31138...,the shoot apic meristem sam dome shape collect...
658,Bruce Shapiro,"Shapiro, Bruce & Finney, Andrew & Hucka, Micha...",SBML models and MathSBML,Chapter,"MathSBML is an open-source, freely downloadabl...",NaN,NaN,· January 2007,Bruce ShapiroAndrew FinneyMichael HuckaBenjami...,NaN,...,NaN,NaN,NaN,NaN,book,NaN,NaN,NaN,https://www.researchgate.net/publication/22660...,mathsbml open sourc freeli download mathematic...
659,Bruce Shapiro,"S Najdi, Tarek & Yang, Chin-Rang & Shapiro, Br...",Application of a Generalized MWC Model for the...,Article,In our effort to elucidate the systems biology...,NaN,NaN,Journal of Bioinformatics and Computational Bi...,Tarek S NajdiChin-Rang YangBruce ShapiroG. Wes...,NaN,...,NaN,NaN,NaN,NaN,10.1109/CSB.2005.15,NaN,NaN,NaN,https://www.researchgate.net/publication/69647...,in effort elucid system biolog model organ esc...
660,Bruce Shapiro,"Finney, Andrew & Hucka, Michael & J Bornstein,...",Software Infrastructure for Effective Communic...,Article,"Until recently, the majority of computational ...",NaN,NaN,· March 2

In [243]:
abstracts = citations_with_abstracts['cleaned'].tolist()

In [244]:
abstracts[0]

'with pressur increas graduat rate reduc time degre higher educ import identifi risk student earli autom earli warn system therefor high desir in paper use unsupervis cluster techniqu predict graduat status declar major five depart california state univers northridg csun base minim number lower divis cours major in addit use detect cluster identifi hidden bottleneck cours'

Now I need to get something to help verify the results.

I'll use the research interests i got from researchgate to administer the test

In [245]:
shapiro = rg_profiles[rg_profiles['faculty'] == "Bruce Shapiro"]

In [297]:
research_interests = shapiro['research'].tolist()[0]
#research_interests = research_interests.split(' ')
print(research_interests,len(research_interests))

Numerical Modeling Bioinformatics and Computational Biology Bioinformatics Molecular Biotechnology Computational Biology Differential Equations Developmental Biology Systems Biology  182


In [299]:
x = np.array(citations_with_abstracts['cleaned'].tolist())
x.reshape(1,45)
print(x.shape)
y = np.array([np.array(research_interests) for i in range(45)])
print(y.shape)
y = y.reshape(45,1)
mlb = MultiLabelBinarizer()
y_enc = mlb.fit_transform(y)
print(y_enc.shape)

(45,)
(45,)
(45, 1)


In [300]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [301]:
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1,2), stop_words="english", sublinear_tf=True)),
                    ('chi', SelectKBest(chi2, k='all')),
                    ('clf', LinearSVC(C=1.0, penalty='l1', max_iter=3000,dual=False))])

In [302]:
model = pipeline.fit(X_train, y_train)

/home/frostburn/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 'Numerical Modeling Bioinformatics and Computational Biology Bioinformatics Molecular Biotechnology Computational Biology Differential Equations Developmental Biology Systems Biology '